### Import Required Libraries and Set Up Environment Variables

In [8]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
from pandas import json_normalize
import json

In [9]:
# Set environment variables from the .env in the local environment
load_dotenv()

nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")

### Access the New York Times API

In [10]:
# Set the base URL
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'

# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# Build URL
final_url = f"{url}fq={filter_query}&sort={sort}&fl={field_list}&begin_date={begin_date}&end_date={end_date}&api-key={nyt_api_key}"
print(final_url)

https://api.nytimes.com/svc/search/v2/articlesearch.json?fq=section_name:"Movies" AND type_of_material:"Review" AND headline:"love"&sort=newest&fl=headline,web_url,snippet,source,keywords,pub_date,byline,word_count&begin_date=20130101&end_date=20230531&api-key=6oHJcweQ4Cni4OZfQl097iPDmm61nUz0


In [11]:
# Create an empty list to store the reviews
reviews_list = []

# Loop through pages 0-19
for page_num in range(20):
    # Create a query with a page number
    query_url = f"{final_url}&page={str(page_num)}"
    
    # Make a "GET" request and retrieve the JSON
    #response = requests.get(page_num)
    data = requests.get(query_url).json()
    
    # Add a twelve-second interval between queries to stay within API query limits
    time.sleep(12)

    # Try to save the reviews to the reviews_list
    try:
        docs = data.get("response", {}).get("docs", [])
        if not docs:
            print(f"No results found for page {page_num}.")
            break

        for review in docs:
            reviews_list.append(review)
        print(f"Retrieved page {page_num}.")
    except Exception as e:
        print(f"Error processing page {page_num}: {str(e)}")

Retrieved page 0.
Retrieved page 1.
Retrieved page 2.
Retrieved page 3.
Retrieved page 4.
Retrieved page 5.
Retrieved page 6.
Retrieved page 7.
Retrieved page 8.
Retrieved page 9.
Retrieved page 10.
Retrieved page 11.
Retrieved page 12.
Retrieved page 13.
Retrieved page 14.
Retrieved page 15.
Retrieved page 16.
Retrieved page 17.
Retrieved page 18.
Retrieved page 19.


In [12]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(reviews_list[:5], indent=4))

[
    {
        "web_url": "https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html",
        "snippet": "A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.",
        "source": "The New York Times",
        "headline": {
            "main": "\u2018The Attachment Diaries\u2019 Review: Love, Sick",
            "kicker": null,
            "content_kicker": null,
            "print_headline": "The Attachment Diaries",
            "name": null,
            "seo": null,
            "sub": null
        },
        "keywords": [
            {
                "name": "subject",
                "value": "Movies",
                "rank": 1,
                "major": "N"
            },
            {
                "name": "creative_works",
                "value": "The Attachment Diaries (Movie)",
                "rank": 2,
                "major": "N"
            },
            {
                "name": "persons",
 

In [13]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()
reviews_df = pd.DataFrame(reviews_list)

if not reviews_df.empty:
    reviews_df_normalized = json_normalize(reviews_df.to_dict("records"))

In [14]:
# Extract the title from the "headline.main" column and
# save it to a new column "title"
# Title is between unicode characters \u2018 and \u2019. 
# End string should include " Review" to avoid cutting title early
extract_title = lambda st: st[st.find("\u2018")+1:st.find("\u2019 Review")]

reviews_df_normalized["title"] = reviews_df_normalized["headline.main"].apply(extract_title)

print(reviews_df_normalized)
# def extract_title(headline):
#     start_str = "\u2018"
#     end_str = "Review"

#     if start_str in headline and end_str in headline:
#         start_index = headline.find(start_str) + len(start_str)
#         end_index = headline.find(end_str, start_index) + len(end_str)

#         return headline[start_index:end_index]
#     return ""

# reviews_df_normalized["title"] = reviews_df_normalized["headline.main"].apply(extract_title)

# print(reviews_df_normalized)

                                               web_url  \
0    https://www.nytimes.com/2023/05/25/movies/the-...   
1    https://www.nytimes.com/2023/05/04/movies/what...   
2    https://www.nytimes.com/2023/05/04/movies/you-...   
3    https://www.nytimes.com/2023/04/21/movies/a-to...   
4    https://www.nytimes.com/2023/04/20/movies/othe...   
..                                                 ...   
195  https://www.nytimes.com/2017/03/09/movies/the-...   
196  https://www.nytimes.com/2017/03/09/movies/revi...   
197  https://www.nytimes.com/2017/03/02/movies/love...   
198  https://www.nytimes.com/2017/02/16/movies/ever...   
199  https://www.nytimes.com/2017/02/09/movies/kedi...   

                                               snippet              source  \
0    A gynecologist and her patient form a horrifyi...  The New York Times   
1    Two childhood friends navigate cultural differ...  The New York Times   
2    Religion comes between two girls falling in lo...  The New York 

In [15]:
# Extract 'name' and 'value' from items in "keywords" column
def extract_keywords(keyword_list):
    extracted_keywords = ""
    for item in keyword_list:
        # Extract 'name' and 'value'
        keyword = f"{item['name']}: {item['value']};" 
        # Append the keyword item to the extracted_keywords list
        extracted_keywords += keyword
    return extracted_keywords

# Fix the "keywords" column by converting cells from a list to a string
def fix_keywords_column(data):
    for index, row in data.iterrows():
        row["keywords"] = extract_keywords(row["keywords"])
    return data

fixed_data = fix_keywords_column(reviews_df_normalized)

In [16]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database
titles = fixed_data["title"].to_list()

print(titles)

['The Attachment Diaries', 'What’s Love Got to Do With It?’ Probably a Lo', 'You Can Live Forever', 'A Tourist’s Guide to Love', 'Other People’s Children', 'One True Loves', 'The Lost Weekend: A Love Story', 'A Thousand and One', 'Your Place or Mine', 'Love in the Time of Fentanyl', 'Pamela, a Love Story', 'In From the Side', 'After Love', 'Alcarràs', 'Nelly & Nadine', 'Lady Chatterley’s Lover', 'The Sound of Christmas', 'The Inspection', 'Bones and All', 'My Policeman', 'About Fate', 'Waiting for Bojangles', 'I Love My Dad', 'A Love Song', 'Alone Together', 'Art of Love', 'The Wheel', 'Thor: Love and Thunder', 'Both Sides of the Blade', 'Fire of Love', 'Love & Gelato', 'Stay Prayed Up', 'Benediction', 'Dinner in America', 'In a New York Minute', 'Anaïs in Love', 'I Love America', 'See You Then', 'La Mami', 'Love After Love', 'Deep Water', 'Lucy and Desi', 'Cyrano', 'The In Between', 'Book of Love', 'Lingui, the Sacred Bonds', 'The Pink Cloud', 'A Journal for Jordan', 'West Side Story'

### Access The Movie Database API

In [17]:
# Prepare The Movie Database query
url = "https://api.themoviedb.org/3/search/movie?query="
tmdb_key_string = "&api_key=" + tmdb_api_key

In [18]:
# Create an empty list to store the results
tmdb_movies_list = []

# Create a request counter to sleep the requests after a multiple
# of 50 requests
request_counter = 1

# Loop through the titles
for title in titles:
    # Check if we need to sleep before making a request
    if request_counter % 50 == 0:
        print("Application is sleeping.")
        time.sleep(1)

    # Add 1 to the request counter
    request_counter += 1
    
    # Perform a "GET" request for The Movie Database
    try:
        response = requests.get(f"https://api.themoviedb.org/3/search/movie?api_key=9ea3ad4760b7645d55e9f262a0cd2e15&query={title}")

    # Include a try clause to search for the full movie details.
    # Use the except clause to print out a statement if a movie
    # is not found.
        try:
        # Get movie id
            movie_id = response.json()["results"][0]["id"]

        # Make a request for a the full movie details
            movie_response = requests.get(f"https://api.themoviedb.org/3/movie/{movie_id}?api_key=9ea3ad4760b7645d55e9f262a0cd2e15")

        # Execute "GET" request with url
            movie_data = movie_response.json()
        
        # Extract the genre names into a list
            genre_names = [genre["name"] for genre in movie_data["genres"]]

        # Extract the spoken_languages' English name into a list
            spoken_languages = [lang["english_name"] for lang in movie_data["spoken_languages"]]

        # Extract the production_countries' name into a list
            production_countries = [country["name"] for country in movie_data["production_countries"]]

        # Add the relevant data to a dictionary and
        # append it to the tmdb_movies_list list
            movie_info = {
                "title": movie_data.get("title"),
                "original_title": movie_data.get("original_title"),
                "budget": movie_data.get("budget"),
                "original_language": movie_data.get("original_language"),
                "homepage": movie_data.get("homepage"),
                "overview": movie_data.get("overview"),
                "popularity": movie_data.get("popularity"),
                "runtime": movie_data.get("popularity"),
                "revenue": movie_data.get("revenue"),
                "release_date": movie_data.get("release_date"),
                "vote_count": movie_data.get("vote_count"),
                "vote_average": movie_data.get("vote_average"),
                "genres": genre_names,
                "spoken_languages": spoken_languages,
                "production_countries": production_countries
        }
            tmdb_movies_list.append(movie_info)
        # Print out the title that was found
            print(f"Found: {movie_data['title']}")
        except IndexError:
            print(f"Movie '{title}' not found.")
    except requests.exceptions.RequestException as e:
        print("Error making API request", e)


Found: The Attachment Diaries
Movie 'What’s Love Got to Do With It?’ Probably a Lo' not found.
Found: You Can Live Forever
Found: A Tourist's Guide to Love
Found: Other People's Children
Found: One True Loves
Found: The Lost Weekend: A Love Story
Found: A Thousand and One
Found: Your Place or Mine
Found: Love in the Time of Fentanyl
Found: Pamela, A Love Story
Found: In from the Side
Found: After Love
Found: Alcarràs
Found: Nelly and Monsieur Arnaud
Found: Lady Chatterley's Lover
Found: The Sound of Christmas
Found: The Inspection
Found: Bones and All
Found: My Policeman
Found: About Fate
Found: Waiting for Bojangles
Found: I Love My Dad
Found: A Love Song
Found: Alone Together
Found: Art of Love
Found: 2099: The Soldier Protocol
Found: Thor: Love and Thunder
Found: Both Sides of the Blade
Found: Fire of Love
Found: Art of Love
Found: Stay Prayed Up
Found: Benediction
Found: Dinner in America
Found: In a New York Minute
Found: Anaïs in Love
Found: I Love America
Found: See You Then
Fou

In [19]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
preview_data = tmdb_movies_list[:5]
formatted_data =json.dumps(preview_data, indent=4)
print(formatted_data)

[
    {
        "title": "The Attachment Diaries",
        "original_title": "El apego",
        "budget": 0,
        "original_language": "es",
        "homepage": "",
        "overview": "Argentina, 1970s. A desperate young woman goes to a clinic to have a clandestine abortion. As her pregnancy is already through the fourth month, the doctor refuses. Instead, she proposes to sell the baby to one of her clients and offers to provide shelter in her house until the child is born. Their disturbed personalities will become intertwined in a strange and dangerous relationship.",
        "popularity": 5.887,
        "runtime": 5.887,
        "revenue": 0,
        "release_date": "2021-10-07",
        "vote_count": 4,
        "vote_average": 3.0,
        "genres": [
            "Drama",
            "Mystery",
            "Thriller",
            "Horror"
        ],
        "spoken_languages": [
            "Spanish"
        ],
        "production_countries": [
            "Argentina"
        ]

In [20]:
# Convert the results to a DataFrame
tmdb_df = pd.DataFrame(tmdb_movies_list)

### Merge and Clean the Data for Export

In [21]:
# Merge the New York Times reviews and TMDB DataFrames on title
nyt_tmdb_df = pd.merge(fixed_data, tmdb_df, on="title", how="inner")
print(nyt_tmdb_df)

                                               web_url  \
0    https://www.nytimes.com/2023/05/25/movies/the-...   
1    https://www.nytimes.com/2023/05/04/movies/you-...   
2    https://www.nytimes.com/2023/04/13/movies/one-...   
3    https://www.nytimes.com/2023/04/13/movies/the-...   
4    https://www.nytimes.com/2023/03/30/movies/a-th...   
..                                                 ...   
121  https://www.nytimes.com/2017/09/14/movies/in-s...   
122  https://www.nytimes.com/2017/07/27/movies/the-...   
123  https://www.nytimes.com/2017/06/15/movies/lost...   
124  https://www.nytimes.com/2017/03/09/movies/the-...   
125  https://www.nytimes.com/2017/03/09/movies/revi...   

                                               snippet              source  \
0    A gynecologist and her patient form a horrifyi...  The New York Times   
1    Religion comes between two girls falling in lo...  The New York Times   
2    A film adaptation of Taylor Jenkins Reid’s nov...  The New York 

In [22]:
# Remove list brackets and quotation marks on the columns containing lists
# Create a list of the columns that need fixing
columns_to_fix = ["genres", "spoken_languages", "production_countries"]

# Create a list of characters to remove
characters_to_remove = ['[', ']', "'"]

# Loop through the list of columns to fix
for col in columns_to_fix:

    # Convert the column to type 'str'
    nyt_tmdb_df[col] = nyt_tmdb_df[col].astype(str)

    # Loop through characters to remove
    for char in characters_to_remove:
        nyt_tmdb_df[col] = nyt_tmdb_df[col].str.replace(char, '')

# Display the fixed DataFrame
display(nyt_tmdb_df)

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,...,overview,popularity,runtime,revenue,release_date,vote_count,vote_average,genres,spoken_languages,production_countries
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,...,"Argentina, 1970s. A desperate young woman goes...",5.887,5.887,0,2021-10-07,4,3.000,"Drama, Mystery, Thriller, Horror",Spanish,Argentina
1,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,...,"When Jaime, a gay teenager, is sent to live in...",22.259,22.259,15055,2023-03-24,39,6.564,"Drama, Romance","English, French","Canada, United States of America"
2,https://www.nytimes.com/2023/04/13/movies/one-...,A film adaptation of Taylor Jenkins Reid’s nov...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-13T11:00:06+0000,320,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,...,Emma and Jesse are living the perfect life tog...,15.184,15.184,37820,2023-04-07,79,6.500,"Romance, Comedy, Drama","English, Spanish","Czech Republic, United States of America"
3,https://www.nytimes.com/2023/04/13/movies/the-...,There’s not much Lennon music heard in this do...,The New York Times,"[{'name': 'subject', 'value': 'Documentary Fil...",2023-04-13T11:00:03+0000,327,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,...,May Pang lovingly recounts her life in rock & ...,27.210,27.210,77145,2023-04-13,2,6.000,Documentary,English,United States of America
4,https://www.nytimes.com/2023/03/30/movies/a-th...,A mesmerizing Teyana Taylor stars in A.V. Rock...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-03-30T18:53:42+0000,971,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,An Unbending Will Meets a Shifting City,...,Struggling but unapologetically living on her ...,13.225,13.225,3395595,2023-03-31,90,6.872,"Drama, Crime","English, Spanish, Portuguese",United States of America
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121,https://www.nytimes.com/2017/09/14/movies/in-s...,A sheltered young woman enamored of the films ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-09-14T11:00:24+0000,272,Review: Love of Il Maestro Drives ‘In Search o...,None,None,In Search of Fellini,...,"Lucy, a small-town girl from Ohio, discovers t...",9.518,9.518,0,2017-09-15,40,5.800,"Drama, Adventure",English,
122,https://www.nytimes.com/2017/07/27/movies/the-...,"In war-torn Liberia, Charlize Theron and Javie...",The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-07-27T20:29:01+0000,280,Review: Aid Workers in Love and War in Sean Pe...,None,None,The Last Face,...,"Miguel, a heroic Spanish doctor, puts himself ...",19.107,19.107,0,2017-01-11,194,5.549,Drama,"English, Portuguese",United States of America
123,https://www.nytimes.com/2017/06/15/movies/lost...,The filmmakers Fiona Gordon and Dominique Abel...,The New York Times,"[{'name': 'creative_works', 'value': 'Lost in ...",2017-06-15T19:37:09+0000,254,Review: Finding Love (and Slapstick) While ‘Lo...,None,None,Lost in Paris,...,Fiona visits Paris for the first time to assis...,9.949,9.949,0,2017-01-14,114,6.022,Comedy,French,"Belgium, France"
124,https://www.nytimes.com/2017/03/09/movies/the-...,This moody romance stars Tatiana Maslany (“Orp...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-03-09T21:54:58+

In [23]:
# Drop "byline.person" column
nyt_tmdb_df.drop("byline.person", axis=1, inplace=True)

In [24]:
# Delete duplicate rows and reset index
list_cols = ["keywords", "genres", "spoken_languages", "production_countries"]
for col in list_cols:
    nyt_tmdb_df[col] = nyt_tmdb_df[col].astype(str)
    
nyt_tmdb_df.drop_duplicates(inplace=True)
nyt_tmdb_df.reset_index(drop=True, inplace=True)

In [25]:
# Export data to CSV without the index
nyt_tmdb_df.to_csv("output.csv", index=False)